In [10]:
######## Imports ########
import os
import cv2
from PIL import ImageGrab

import matplotlib
matplotlib.use("Qt5agg") 
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

In [11]:
# ######## LOAD MODEL #########
model = model_from_json(open("fer2.json", "r").read()) 
model.load_weights('fer2.h5') #load weights

face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') #load cascade

In [12]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [13]:
######## Global Variables #########
plt.style.use('ggplot')
x_axis = np.arange(100)  # will be used in graphing. 
line1 = [] # will initaite the loop for live plotting.
plot_array = np.full((7,100), 0) #create an array that will be updated with live values.
counter= 1 #used to close every 10th window
max_faces = 4 #sets a max number of faces to be recognized to protect performance.
results_array = np.full((1,8), 0.0, dtype='float64')

In [ ]:
try:
    cap = cv2.VideoCapture(0)

    while True:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_haar_cascade.detectMultiScale(gray, 1.3, 5)
 
        for (x,y,w,h) in faces:
            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
            detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
            detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
 
            img_pixels /= 255
 
            predictions = model.predict(img_pixels)
 
            #find max indexed array
            max_index = np.argmax(predictions[0])
 
            emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion = emotions[max_index]
 
            cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.imshow('img', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
except KeyboardInterrupt:
	cv2.destroyAllWindows()
	
np.savetxt('emotion_results', results_array, delimiter=",")